<img style="float: left;;" src='https://github.com/gdesirena/Procesamiento_Natural_del_Lenguaje_2024/blob/main/Modulo%20III/Figures/alinco.png?raw=1' /></a>

# Modulo II: Traducción automática y Local Sensitive Hashing (LSH)


Ahora implementaremos un sistema de traducción automática y luego
veremos cómo funcionan las funciones hash. Empecemos importando
las funciones requeridas!


```
nltk.download('stopwords')
nltk.download('twitter_samples')
```


In [2]:
import pdb
import pickle
import string

import time

import gensim
import matplotlib.pyplot as plt
import nltk
import numpy as np
import scipy
import sklearn
from gensim.models import KeyedVectors
from nltk.corpus import stopwords, twitter_samples
from nltk.tokenize import TweetTokenizer

from utils import (cosine_similarity, get_dict,
                   process_tweet)
from os import getcwd

In [3]:
# add folder, tmp2, from our local workspace containing pre-downloaded corpora files to nltk's data path
filePath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filePath)

In [4]:
nltk.download('stopwords')
nltk.download('twitter_samples')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.


True

# Embeddings para palabras en Inglés y Frances

Escribe un programa que traduzca del inglés al francés.

## Los datos

El conjunto de datos completo para las wordembeddings en inglés es de aproximadamente 3,64 gigabytes, y el francés
son de aproximadamente 629 megabytes. Trabajaremos con un conjunto más pequeño de datos, una muestra significativa.

* English embeddings from Google code archive word2vec
[look for GoogleNews-vectors-negative300.bin.gz](https://code.google.com/archive/p/word2vec/)
    * You'll need to unzip the file first.
* and the French embeddings from
[cross_lingual_text_classification](https://github.com/vjstark/crosslingual_text_classification).
    * in the terminal, type (in one line)
    `curl -o ./wiki.multi.fr.vec https://dl.fbaipublicfiles.com/arrival/vectors/wiki.multi.fr.vec`

copiar y pegar el código para descargar todos los datos

```python

from gensim.models import KeyedVectors

en_embeddings = KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary = True)
fr_embeddings = KeyedVectors.load_word2vec_format('./wiki.multi.fr.vec')


# loading the english to french dictionaries
en_fr_train = get_dict('en-fr.train.txt')
print('The length of the english to french training dictionary is', len(en_fr_train))
en_fr_test = get_dict('en-fr.test.txt')
print('The length of the english to french test dictionary is', len(en_fr_train))

english_set = set(en_embeddings.vocab)
french_set = set(fr_embeddings.vocab)
en_embeddings_subset = {}
fr_embeddings_subset = {}
french_words = set(en_fr_train.values())

for en_word in en_fr_train.keys():
    fr_word = en_fr_train[en_word]
    if fr_word in french_set and en_word in english_set:
        en_embeddings_subset[en_word] = en_embeddings[en_word]
        fr_embeddings_subset[fr_word] = fr_embeddings[fr_word]


for en_word in en_fr_test.keys():
    fr_word = en_fr_test[en_word]
    if fr_word in french_set and en_word in english_set:
        en_embeddings_subset[en_word] = en_embeddings[en_word]
        fr_embeddings_subset[fr_word] = fr_embeddings[fr_word]


pickle.dump( en_embeddings_subset, open( "en_embeddings.p", "wb" ) )
pickle.dump( fr_embeddings_subset, open( "fr_embeddings.p", "wb" ) )
```

#### Un subconjunto de datos

Para realizar esta actividad, usaremos un subconjunto de datos que contenga algunos word embeddings.

In [6]:
en_embeddings_subset = pickle.load(open('en_embeddings.p', 'rb'))
fr_embeddings_subset = pickle.load(open('fr_embeddings.p', 'rb'))

In [7]:
type(en_embeddings_subset)

dict

In [9]:
len(en_embeddings_subset['king'])

300

In [12]:
len(fr_embeddings_subset['génie'])

300

#### Observando los datos

* en_embeddings_subset: el key es una palabra en inglés, y el valor es un arreglo de 300 elementos, que representa el wordembedding de la palabra.
```
'the': array([ 0.08007812,  0.10498047,  0.04980469,  0.0534668 , -0.06738281, ....
```

* fr_embeddings_subset: el key es una palabra en francés,y el valor es un arreglo de 300 elementos, que representa el wordembedding de la palabra.
```
'la': array([-6.18250e-03, -9.43867e-04, -8.82648e-03,  3.24623e-02,...
```

#### Cargue dos diccionarios que mapean las palabras del inglés al francés
* Un diccionario de entrenamiento
* y un diccionario de prueba.


In [13]:
import pandas as pd
def get_dic(file_name):
  my_file = pd.read_csv(file_name, delimiter='')
  etof = {}
  for i in range(len(my_file)):
    en = my_file.loc[i][0]
    fr = my_file.loc[i][1]
    etof[en] = fr
  return etof


In [14]:
#Cargar los diccionarios de inglés a francés
en_fr_train = get_dict('en-fr.train.txt')
en_fr_test = get_dict('en-fr.test.txt')

In [15]:
en_fr_train

{'the': 'la',
 'and': 'et',
 'was': 'était',
 'for': 'pour',
 'that': 'cela',
 'with': 'avec',
 'from': 'depuis',
 'this': 'ce',
 'utc': 'tuc',
 'his': 'son',
 'not': 'pas',
 'are': 'sont',
 'talk': 'parlez',
 'which': 'lequel',
 'also': 'egalement',
 'were': 'étaient',
 'but': 'mais',
 'have': 'ont',
 'one': 'one',
 'new': 'nouveautés',
 'first': 'premiers',
 'page': 'page',
 'you': 'you',
 'they': 'eux',
 'had': 'avais',
 'article': 'article',
 'who': 'who',
 'all': 'all',
 'their': 'leurs',
 'there': 'là',
 'made': 'fabriqué',
 'its': 'son',
 'people': 'personnes',
 'may': 'peut',
 'after': 'aprés',
 'other': 'autres',
 'should': 'devrais',
 'two': 'deux',
 'score': 'partition',
 'her': 'her',
 'can': 'peut',
 'would': 'ferait',
 'more': 'plus',
 'she': 'elle',
 'when': 'quand',
 'time': 'heure',
 'team': 'equipe',
 'american': 'américains',
 'such': 'telles',
 'discussion': 'débat',
 'links': 'liens',
 'only': 'seule',
 'some': 'quelques',
 'see': 'vois',
 'united': 'unies',
 'year

#### Viendo los diccionarios de Inglés y Francés

* `en_fr_train` es un diccionario donde el key es la palabra en inglés y el valor es la palabra en francés.
```
{'the': 'la',
 'and': 'et',
 'was': 'était',
 'for': 'pour',
```

* `en_fr_test` es similar que `en_fr_train`, pero este es un conjunto de prueba.

## Generando matrices de embeddings y transformación

####  Traducción de diccionario de inglés a francés mediante embeddings

Implementaremos la función `get_matrices`, que toma los datos cargados y retorna las matrices `X` y `Y`.

Entrada:
- `en_fr` : diccionario del Inglés a Francés
- `en_embeddings` : embeddings de palabras en inglés
- `fr_embeddings` : embeddings de palabras en Francés

Retorna:
- matrices `X` y `Y`, donde cada renglón en X es la palabra embebida para una plabra en inglés, y lo mismo con Y es la palabra embebida en francés.

<div style="width:image width px; font-size:100%; text-align:center;">
<img src='https://github.com/gdesirena/Procesamiento_Natural_del_Lenguaje_2024/blob/main/Modulo%20III/Figures/X_to_Y.jpg?raw=1' alt="alternate text" width="width" height="height" style="width:800px;height:200px;" /> Figure 2 </div>

Utilice el diccionario `en_fr` para asegurarse de que la i-ésima fila de la matriz` X`
corresponde a la i-ésima fila de la matriz "Y".


In [17]:
def get_matrices(en_fr, french_vecs, english_vecs):
  #Crear dos listas X_l, Y_l listas para inglés y francés
  X_l = list()
  Y_l = list()

  #Obtenemos las palabras de inglés
  english_set = english_vecs.keys()
  french_set = french_vecs.keys()

  #Guardar las palabras en francés que son parte del diccionario inglés y francés
  french_words = set(en_fr.values())

  #Ciclo para las palabras en inglés y francés en el diccionario
  for en_word, fr_word in en_fr.items():
     #checar qie la palabra en francés tiene un vector embedding y que la palabra en inglés también tenga un vec embedding
      if (fr_word in french_set ) and (en_word in english_set):
        #obtener los vectores embeddings de la palabra
        en_vec = english_vecs[en_word]
        fr_vec = french_vecs[fr_word]

        #Guardar los embeddings de las palabras en las listas correspondientes a inglés y francés
        X_l.append(en_vec)
        Y_l.append(fr_vec)
    #convertir la lista de vectores embeddings a una matriz
  X = np.vstack(X_l)
  Y = np.vstack(Y_l)

  return X, Y





Ahora usaremos la función `get_matrices ()` para obtener los conjuntos `X_train` e` Y_train`
de los embeddings de palabras en inglés y francés.


In [18]:
X_train, Y_train = get_matrices(en_fr_train, fr_embeddings_subset, en_embeddings_subset)

In [20]:
len(X_train), len(Y_train)

(4932, 4932)

In [22]:
X_train.shape, Y_train.shape

((4932, 300), (4932, 300))


# Traductores

<div style="width:image width px; font-size:100%; text-align:center;"><img src='https://github.com/gdesirena/Procesamiento_Natural_del_Lenguaje_2024/blob/main/Modulo%20III/Figures/e_to_f.jpg?raw=1' alt="alternate text" width="width" height="height" style="width:700px;height:200px;" />  </div>

Escriba un programa que traduzca palabras del inglés al francés utilizando word embeddings y modelos de espacio vectorial.


##  Traducción como transformación lineal de embeddings
Dados los diccionarios de embeddings de palabras en inglés y francés, crearemos una matriz de transformación `R`
* Dada una palabra incrustada en inglés, $ \mathbf {e} $, puede multiplicar $ \mathbf {eR} $ para obtener una nueva palabra incrustada $ \mathbf {f} $.

    
* A continuación, puede calcular los vecinos más cercanos a `f` en los embeddings francesas y recomendar la palabra que es más similar a los embeddings de palabras transformadas.

### Describir la traducción como el problema de minimización

Encuentre una matriz `R` que minimice la siguiente ecuación.

$$\arg \min _{\mathbf{R}}\| \mathbf{X R} - \mathbf{Y}\|_{F}\tag{1} $$

### Norma de Frobenius

La norma de Frobenius de una matriz $ A $ (asumiendo que es de dimensión $ m, n $) se define como la raíz cuadrada de la suma de los cuadrados absolutos de sus elementos:

$$\|\mathbf{A}\|_{F} \equiv \sqrt{\sum_{i=1}^{m} \sum_{j=1}^{n}\left|a_{i j}\right|^{2}}\tag{2}$$


### Función de perdida

En las aplicaciones del mundo real, la pérdida de la norma Frobenius:

$$\| \mathbf{XR} - \mathbf{Y}\|_{F}$$

a menudo se reemplaza por su valor al cuadrado dividido por $ m $:

$$ \frac{1}{m} \|  \mathbf{X R} - \mathbf{Y} \|_{F}^{2}$$

donde $ m $ es el número de ejemplos (filas en $ \mathbf {X} $).

* Se encuentra la misma R cuando se usa esta función de pérdida en comparación con la norma de Frobenius original.
* La razón para tomar el cuadrado es que es más fácil calcular el gradiente del Frobenius al cuadrado.
* La razón para dividir entre $ m $ es que estamos más interesados en la pérdida promedio por inserción que en la pérdida de todo el conjunto de entrenamiento.
     * La pérdida de todo el conjunto de entrenamiento aumenta con más palabras (ejemplos de entrenamiento),
     por lo que tomar el promedio nos ayuda a rastrear la pérdida promedio independientemente del tamaño del conjunto de entrenamiento.



### Implementación del mecanismo de traducción

#### Calculando el loss
* La función de pérdida será la norma de Frobenoius al cuadrado de la diferencia entre
matriz y su aproximación, dividida por el número de ejemplos de entrenamiento $ m $.
* Su fórmula es:
$$ L(X, Y, R)=\frac{1}{m}\sum_{i=1}^{m} \sum_{j=1}^{n}\left( a_{i j} \right)^{2}$$

donde $a_{i j}$ es el valo de la $i$-ésimo renglón y $j$-ésima columna de la matriz $\mathbf{XR}-\mathbf{Y}$.

#### crear la función `compute_loss()`
* Calcular la aproximación de `Y` mediante la matriz multiplicando` X` y `R`
* Calcular la diferencia `XR - Y`
* Calcular la norma de Frobenius al cuadrado de la diferencia y divídala por $ m $.


In [23]:
def compute_loss(X,Y,R):
  m = X.shape[0]
  loss = np.dot(X,R) - Y
  loss2 = loss**2
  sum_loss2 = np.sum(loss2)

  L = sum_loss2/m

  return L





### Calculando el gradiente de la función loss con respecto a la matriz de transforación R

* Calcular el gradiente de la pérdida con respecto a la matriz de transformación "R".
* El gradiente nos da la dirección en la que debemos disminuir `R`
para minimizar la pérdida.
* $ m $ es el número de ejemplos de entrenamiento (número de filas en $ X $).
* La fórmula para el gradiente de la función de pérdida $ 𝐿 (𝑋, 𝑌, 𝑅) $ es:

$$\frac{d}{dR}𝐿(𝑋,𝑌,𝑅)=\frac{d}{dR}\Big(\frac{1}{m}\| X R -Y\|_{F}^{2}\Big) = \frac{2}{m}X^{T} (X R - Y)$$



In [24]:
def compute_gradient(X,Y,R):
  m=X.shape[0]
  gradient = np.dot(X.transpose(), np.dot(X,R)-Y)*(2/m)
  return gradient


### Encontrar la R óptima con el algoritmo de descenso de gradiente

#### Gradient descent


Pseudocódigo:
1. Calcular el gradiente $g$ del loss con respecto a la matriz $R$.
2. Update $R$ con la formula:
$$R_{\text{new}}= R_{\text{old}}-\alpha g$$

Donde $\alpha$ es el learning rate, que es un escalar.

#### Learning rate

* La tasa de aprendizaje o "tamaño de paso" $ \ alpha $ es un coeficiente que decide cuánto queremos cambiar $ R $ en cada paso.
* Si cambiamos $ R $ demasiado, podríamos saltarnos el óptimo dando un paso demasiado grande.
* Si solo hacemos pequeños cambios en $ R $, necesitaremos muchos pasos para alcanzar el óptimo.
* La tasa de aprendizaje $ \ alpha $ se usa para controlar esos cambios.
* Los valores de $ \ alpha $ se eligen dependiendo del problema, y usaremos `learning_rate` $ = 0.0003 $ como valor predeterminado para nuestro algoritmo.

In [26]:
X_train.shape

(4932, 300)

#### Implementar la función `align_embeddings()`

In [27]:
def align_embeddings(X,Y, train_steps=100, learning_rate=0.0003):
  np.random.seed(0)
  R = np.random.rand(X.shape[1], X.shape[1]) #Matriz cuadrada de 300x300

  for i in range(train_steps):
    if i%25==0:
      print(f'La pérdida en la iteración {i} es {compute_loss(X,Y,R):.4f}')
    gradiente = compute_gradient(X,Y,R)

    #Actualizar R
    R = R - learning_rate*gradiente

  return R

In [28]:
np.random.seed(129)
m = 10
n = 5
X = np.random.rand(m,n)
Y = np.random.rand(m,n)
R = align_embeddings(X,Y)

La pérdida en la iteración 0 es 4.5591
La pérdida en la iteración 25 es 4.4492
La pérdida en la iteración 50 es 4.3421
La pérdida en la iteración 75 es 4.2379


In [30]:
R.shape

(5, 5)

## Calcular la matriz de transformación R

Usando el conjunto de entrenamiento, encuentre la matriz de transformación $\mathbf{R}$ llamando a la función `align_embeddings()`.

**NOTA:** La siguiente celda de código tardará unos minutos en ejecutarse por completo (~3 minutos).

In [31]:
X_train.shape

(4932, 300)

In [33]:
align_embeddings(X_train,Y_train,train_steps=500, learning_rate=0.8)

La pérdida en la iteración 0 es 957.9498
La pérdida en la iteración 25 es 97.4537
La pérdida en la iteración 50 es 26.7613
La pérdida en la iteración 75 es 9.7934
La pérdida en la iteración 100 es 4.3927
La pérdida en la iteración 125 es 2.3407
La pérdida en la iteración 150 es 1.4567
La pérdida en la iteración 175 es 1.0395
La pérdida en la iteración 200 es 0.8288
La pérdida en la iteración 225 es 0.7169
La pérdida en la iteración 250 es 0.6549
La pérdida en la iteración 275 es 0.6195
La pérdida en la iteración 300 es 0.5987
La pérdida en la iteración 325 es 0.5862
La pérdida en la iteración 350 es 0.5785
La pérdida en la iteración 375 es 0.5737
La pérdida en la iteración 400 es 0.5707
La pérdida en la iteración 425 es 0.5687
La pérdida en la iteración 450 es 0.5674
La pérdida en la iteración 475 es 0.5665


array([[-0.01094974, -0.01264808,  0.00060607, ...,  0.00090619,
        -0.00317652,  0.00120469],
       [ 0.01800591,  0.00169838,  0.0030341 , ...,  0.02487439,
        -0.00213227,  0.00160555],
       [ 0.00575546,  0.00059365,  0.01488434, ..., -0.00164987,
        -0.00653721,  0.00537056],
       ...,
       [-0.00174451,  0.00417253, -0.01830601, ..., -0.00990938,
         0.00581949,  0.00226011],
       [ 0.00442384, -0.00485468, -0.00123269, ..., -0.00439163,
        -0.00102191,  0.0011057 ],
       [ 0.00557851, -0.01249637, -0.00374668, ..., -0.01588083,
         0.00900329, -0.00426499]])


## Probando el traductor

### Algoritmo k-Nearest neighbors

[k-Nearest neighbors algorithm](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm)
* k-NN es un método que toma un vector como entrada y encuentra los otros vectores en el conjunto de datos que están más cerca de él.
* La 'k' es el número de "vecinos más cercanos" a encontrar (por ejemplo, k = 2 encuentra los dos vecinos más cercanos).

### Buscando el embedding de la traducción
Dado que estamos aproximando la función de traducción de los embeddings de inglés a francés mediante una matriz de transformación lineal $ \mathbf {R} $, la mayoría de las veces no obtendremos la incrustación exacta de una palabra francesa cuando transformamos los embeddings $ \mathbf { e} $ de alguna palabra en inglés en particular en el espacio de embeddings francés.
* ¡Aquí es donde $ k $ -NN se vuelve realmente útil! Al usar $ 1 $ -NN con $ \mathbf {eR} $ como entrada, podemos buscar un embedding $ \mathbf {f} $ (como una fila) en la matriz $ \mathbf {Y} $ que es la más cercana a el vector transformado $ \mathbf {eR} $

### Similaridad por  Coseno
Similitud de coseno entre los vectores $ u $ y $ v $ calculada como el coseno del ángulo entre ellos.
La formula es
$$\cos(u,v)=\frac{u\cdot v}{\left\|u\right\|\left\|v\right\|}$$

* $\cos(u,v)$ = $1$ cuando $u$ y $v$ se encuentran en la misma línea y tienen la misma dirección.
* $\cos(u,v)$ es $-1$ Cuando ellas tienen direcciones exactamente opuestas.
* $\cos(u,v)$ es $0$ cuando los vectores son ortogonales (perpendiculares) entre sí.

#### Nota: La distancia y la similitud son cosas bastante opuestas.
* Podemos obtener la métrica de la distancia a partir de la similitud del coseno, pero la similitud del coseno no se puede usar directamente como la métrica de la distancia.
* Cuando la similitud del coseno aumenta (hacia $ 1 $), la "distancia" entre los dos vectores disminuye (hacia $ 0 $).
* Podemos definir la distancia del coseno entre $ u $ y $ v $ como
$$ d_{\text {cos}} (u, v) = 1- \cos(u, v) $$

In [38]:
#cosine_similarity
def cosine_similarity(A,B):
  dot = np.dot(A,B)
  norma = np.linalg.norm(A)
  normb = np.linalg.norm(B)
  cos = dot/(norma*normb)
  return cos

**Crear la función**`nearest_neighbor()`

Inputs:
* Vector `v`,
* Conjunto de posibles vecinos `candidates`
* `k` vecinos a encontrar.
* La métrica de distancia a utilizar (sim coseno).
* `cosine_similarity` La función de similaridad.


In [39]:
def nearest_neighbor(v, candidates, k=1):
  similarity_l = []
  for row in candidates:
    cos_similarity = cosine_similarity(v,row)
    similarity_l.append(cos_similarity)
  sorted_ids = np.argsort(similarity_l)

  k_idx = sorted_ids[-k:]

  return k_idx



In [40]:
# Test de la implementación:
v = np.array([1,0,1])
candidates = np.array([[1,0,5],[-2,5,3], [2,0,1], [6,-9,5], [9,9,9]])
print(nearest_neighbor(v, candidates, k=1))

[2]


In [44]:
candidates

array([[ 1,  0,  5],
       [-2,  5,  3],
       [ 2,  0,  1],
       [ 6, -9,  5],
       [ 9,  9,  9]])

In [43]:
candidates[nearest_neighbor(v, candidates, k=3)]

array([[9, 9, 9],
       [1, 0, 5],
       [2, 0, 1]])

### Porbando la traducción y calculando el accuracy

Crearemos la función `test_vocabulary` que mapea en inglés la matriz $X$, matriz $Y$ y $R$ y devuelve el accuracy de las traducciones de $X$ a $Y$ por $R$.

* Calculando el accuracy como $$\text{accuracy}=\frac{\#(\text{correct predictions})}{\#(\text{total predictions})}$$


Veamos cómo funciona su mecanismo de traducción con los datos invisibles:

Logramos traducir palabras de un idioma a otro.
sin siquiera verlos con casi un 56% de precisión usando algunos conceptos básicos como:
¡álgebra lineal y aprender a mapear palabras de un idioma a otro!


# LSH y búsqueda de documentos

* Procesar los tweets y representar cada tweet como un vector (representar un
documento con un embedding vector).
* Utilice hashing y k vecinos más cercanos para encontrar tweets
que son similares a un tweet determinado.


In [ ]:
# tweets positivos y negativos
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')
all_tweets = all_positive_tweets + all_negative_tweets


###  Obteniendo los embeddings de los documentos

#### Modelo de Bag-of-words (BOW)

Los documentos de texto son secuencias de palabras.
* El orden de las palabras marca la diferencia. Por ejemplo, las frases "La tarta de manzana es mejor que la pizza de pepperoni "y" La pizza de pepperoni es mejor que la tarta de manzana "
tienen significados opuestos debido al orden de las palabras.
* Sin embargo, para algunas aplicaciones, ignorar el orden de las palabras puede permitir
nosotros para formar un modelo eficiente y aún eficaz.
* Este enfoque se denomina modelo de documento de bolsa de palabras.


In [ ]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
import re

In [ ]:
def process_tweet(tweet):

    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')

    tweet = re.sub(r'\$\w*', '', tweet)

    tweet = re.sub(r'^RT[\s]+', '', tweet)

    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)

    tweet = re.sub(r'#', '', tweet)

    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and
            word not in string.punctuation):
            stem_word = stemmer.stem(word)
            tweets_clean.append(stem_word)

    return tweets_clean


Crear la función `get_document_embedding()`.
* La función `get_document_embedding()` codifica el documento completo como un embedding del "documento".
* Tomar un documento (como una cadena) y un diccionario, `en_embeddings`
* Procesar el documento y buscar el embedding correspondiente de cada palabra.
* Luego sumar y devuelver la suma de todos los vectores de palabras de ese tweet procesado.

In [ ]:
# testear la función
custom_tweet = "RT @Twitter @chapagain Hello There! Have a great day. :) #good #morning http://chapagain.com.np"


#### Guardar los vectores de documento en un diccionario `get_document_vecs()`

In [ ]:
# UNQ_C14 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def get_document_vecs(all_docs, en_embeddings):

    return document_vec_matrix, ind2Doc_dict



## Buscando los tweets

Ahora tenemos un vector de dimensión (m, d) donde `m` es el número de tweets
(10,000) y `d` es la dimensión de los embeddings (300).


## Buscando los tweets más similares con LSH

Ahora implementaremos el hashing (LSH) para identificar el tweet más similar.
* En lugar de mirar los 10,000 vectores, puede buscar un subconjunto para encontrar
sus vecinos más cercanos.


<div style="width:image width px; font-size:100%; text-align:center;"><img src='https://github.com/gdesirena/Procesamiento_Natural_del_Lenguaje_2024/blob/main/Modulo%20III/Figures/one.png?raw=1' alt="alternate text" width="width" height="height" style="width:400px;height:200px;" /> Figure 3 </div>

Puede dividir el espacio vectorial en regiones y buscar dentro de una región los vecinos más cercanos de un vector dado.

<div style="width:image width px; font-size:100%; text-align:center;"><img src='https://github.com/gdesirena/Procesamiento_Natural_del_Lenguaje_2024/blob/main/Modulo%20III/Figures/four.png?raw=1' alt="alternate text" width="width" height="height" style="width:400px;height:200px;" /> Figure 4 </div>


#### Escogiendo los número de planos

* Cada plano divide el espacio en $ 2 $ partes.
* Entonces $ n $ planos dividen el espacio en $ 2 ^ {n} $ hash.
* Queremos organizar 10,000 vectores de documentos en depósitos para que cada depósito tenga vectores de $ ~ 16 $.
* Para eso necesitamos $ \frac {10000} {16} = 625 $ buckets.
* Estamos interesados en $ n $, número de aviones, por lo que $ 2 ^ {n} = 625 $. Ahora, podemos calcular $ n = \log_{2} 625 = 9.29 \approx 10 $.

## Obteniendo el número para el vector hash

Para cada vector, necesitamos obtener un número único asociado a ese vector para poder asignarlo a un "bucket de hash".

### Hyperlanes in vector spaces
* En un espacio vectorial dimensional de $ 3 $, el hiperplano es un plano regular. En el espacio vectorial dimensional de $ 2 $, el hiperplano es una línea.
* Generalmente, el hiperplano es un subespacio que tiene una dimensión $ 1 $ menor que la del espacio vectorial original.
* Un hiperplano se define de forma única por su vector normal.
* El vector normal $ n $ del plano $ \ pi $ es el vector al que todos los vectores en el plano $ \ pi $ son ortogonales (perpendiculares en el caso dimensional de $ 3 $).

### Usando Hiperplanos para cortar el espacio vectorial

Podemos usar un hiperplano para dividir el espacio vectorial en $ 2 $ partes.
* Todos los vectores cuyo producto escalar con el vector normal de un plano es positivo están en un lado del plano.
* Todos los vectores cuyo producto escalar con el vector normal del plano es negativo están en el otro lado del plano.

### Encodeando los buckets hash
* Para un vector, podemos tomar su producto escalar con todos los planos, luego codificar esta información para asignar el vector a un solo cubo hash.
* Cuando el vector apunta al lado opuesto del hiperplano de lo normal, codifíquelo con 0.
* De lo contrario, si el vector está en el mismo lado que el vector normal, codifíquelo por 1.
* Si calcula el producto escalar con cada plano en el mismo orden para cada vector, ha codificado el ID de hash único de cada vector como un número binario, como [0, 1, 1, ... 0].


## Obteniendo el número para el vector hash

Para cada vector, necesitamos obtener un número único asociado a ese vector para poder asignarlo a un "bucket de hash".

### Hiperplanos en espacios de vectores
* En un espacio vectorial dimensional de $ 3 $, el hiperplano es un plano regular. En el espacio vectorial dimensional de $ 2 $, el hiperplano es una línea.
* Generalmente, el hiperplano es un subespacio que tiene una dimensión $ 1 $ menor que la del espacio vectorial original.
* Un hiperplano se define de forma única por su vector normal.
* El vector normal $ n $ del plano $ \ pi $ es el vector al que todos los vectores en el plano $ \ pi $ son ortogonales (perpendiculares en el caso dimensional de $ 3 $).

### Usando Hiperplanos para cortar el espacio vectorial

Podemos usar un hiperplano para dividir el espacio vectorial en $ 2 $ partes.
* Todos los vectores cuyo producto escalar con el vector normal de un plano es positivo están en un lado del plano.
* Todos los vectores cuyo producto escalar con el vector normal del plano es negativo están en el otro lado del plano.

### Encodeando los buckets hash
* Para un vector, podemos tomar su producto escalar con todos los planos, luego codificar esta información para asignar el vector a un solo cubo hash.
* Cuando el vector apunta al lado opuesto del hiperplano de lo normal, codifíquelo con 0.
* De lo contrario, si el vector está en el mismo lado que el vector normal, codifíquelo por 1.
* Si calcula el producto escalar con cada plano en el mismo orden para cada vector, ha codificado el ID de hash único de cada vector como un número binario, como [0, 1, 1, ... 0].


### Implementando los hash buckets

Inicializamos los hash de la tabla. Es una lista de matrices `N_UNIVERSES`, cada una describe su propia tabla hash. Cada matriz tiene filas `N_DIMS` y columnas` N_PLANES`. Cada columna de esa matriz es un vector normal dimensional `N_DIMS` para cada uno de los hiperplanos` N_PLANES` que se utilizan para crear cubos de la tabla hash particular.

* Primero multiplica tu vector `v`, con un plano correspondiente. Esto le dará un vector de dimensión $ (1, \text{N_planes}) $.
* Luego, convertirá todos los elementos de ese vector en 0 o 1.
* Creas un vector hash haciendo lo siguiente: si el elemento es negativo, se convierte en un 0; de lo contrario, lo cambias a un 1.
* Luego calcula el número único para el vector iterando sobre `N_PLANES`
* Luego multiplica $ 2^i $ por el bit correspondiente (0 o 1).
* Luego almacenará esa suma en la variable `hash_value`.

Cree un hash para el vector en la siguiente función.
Utilice esta fórmula:

$$ hash = \sum_{i=0}^{N-1} \left( 2^{i} \times h_{i} \right) $$


#### Crea los conjuntos de planos
* Cree múltiples (25) conjuntos de planos (los planos que dividen la región).
* Puede pensar en estos como 25 formas distintas de dividir el espacio vectorial con un conjunto diferente de planos.
* Cada elemento de esta lista contiene una matriz con 300 filas (la palabra vector tiene 300 dimensiones) y 10 columnas (hay 10 planos en cada "universo").


## Creando la Tabla Hash

Dado que ya tenemos una representación para cada vector (o tweet), ahora crearemos una tabla hash. Necesitamos una tabla hash, de modo que, dado un hash_id, pueda buscar rápidamente los vectores correspondientes. Esto le permite reducir su búsqueda por una cantidad de tiempo significativa.

<div style="width:image width px; font-size:100%; text-align:center;"><img src='https://github.com/gdesirena/Procesamiento_Natural_del_Lenguaje_2024/blob/main/Modulo%20III/Figures/table.png?raw=1' alt="alternate text" width="width" height="height" style="width:500px;height:200px;" />  </div>




<a name="3-6"></a>

### Creando todas las tablas hash

Ahora podemos aplicar la función hash a los vectores y almacenarlos en una tabla hash que
le permitiría buscar rápidamente vectores similares.


### Aproximar K-NN


Implementar aproximadamente K vecinos más cercanos utilizando hash sensible a la localidad,
para buscar documentos que sean similares a un documento dado en el
índice `doc_id`.

##### entradas
* `doc_id` es el índice de la lista de documentos` all_tweets`.
* `v` es el vector de documento para el tweet en` all_tweets` en el índice `doc_id`.
* `planes_l` es la lista de planos (la variable global creada anteriormente).
* `k` es el número de vecinos más cercanos a buscar.
* `num_universes_to_use`: para ahorrar tiempo, podemos usar menos que el total
número de universos disponibles. De forma predeterminada, está configurado en `N_UNIVERSES`,
que es $ 25 $ para esta asignación.

La función `approx_knn` encuentra un subconjunto de vectores candidatos que
están en el mismo "depósito de hash" que el vector de entrada 'v'. Entonces se realiza
los k vecinos más cercanos habituales buscan en este subconjunto (en lugar de buscar
a través de los 10,000 tweets).